In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import balanced_accuracy_score



In [3]:
# Loading and preprocessing Data

In [4]:
# Loading data
file_path = Path("../EDA/Resources/cleaned_close1.csv")
df_equities = pd.read_csv(file_path)
df_equities.head()

,symbol,date,high,iexClose,industry,low,sector,volume,death,deathIncrease,hospitalizedIncrease,hospitalizedCurrently,negative,negativeIncrease,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease,p_status
0,PJUL,8/19/2020,27.84,27.710,Investment Trusts/Mutual Funds,27.76,Miscellaneous,3283,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
1,PIM,8/19/2020,4.26,4.240,Investment Trusts/Mutual Funds,4.23,Miscellaneous,31824,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,0
2,PJT,8/19/2020,59.67,58.885,Investment Banks/Brokers,58.60,Finance,23896,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
3,PKBK,8/19/2020,13.46,13.460,Regional Banks,12.89,Finance,2365,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
4,PICK,8/19/2020,28.49,28.380,Investment Trusts/Mutual Funds,28.25,Miscellaneous,71208,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,Null


In [5]:
df_equities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631360 entries, 0 to 631359
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   symbol                    631360 non-null  object 
 1   date                      631360 non-null  object 
 2   high                      631360 non-null  float64
 3   iexClose                  631360 non-null  float64
 4   industry                  631360 non-null  object 
 5   low                       631360 non-null  float64
 6   sector                    631360 non-null  object 
 7   volume                    631360 non-null  int64  
 8   death                     631360 non-null  int64  
 9   deathIncrease             631360 non-null  int64  
 10  hospitalizedIncrease      631360 non-null  int64  
 11  hospitalizedCurrently     631360 non-null  int64  
 12  negative                  631360 non-null  int64  
 13  negativeIncrease          631360 non-null  i

In [6]:

# Drop the null columns where all values are null
df_equities = df_equities.dropna(axis='columns', how='all')

# Drop the null rows
df_equities = df_equities.dropna(how='any')
df_equities.drop(df_equities.loc[df_equities['p_status']== "Null"].index, inplace=True)
df_equities = df_equities.drop(columns = ['symbol','date','industry','sector'])

In [7]:
df_equities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 570400 entries, 0 to 631359
Data columns (total 15 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   high                      570400 non-null  float64
 1   iexClose                  570400 non-null  float64
 2   low                       570400 non-null  float64
 3   volume                    570400 non-null  int64  
 4   death                     570400 non-null  int64  
 5   deathIncrease             570400 non-null  int64  
 6   hospitalizedIncrease      570400 non-null  int64  
 7   hospitalizedCurrently     570400 non-null  int64  
 8   negative                  570400 non-null  int64  
 9   negativeIncrease          570400 non-null  int64  
 10  positive                  570400 non-null  int64  
 11  positiveIncrease          570400 non-null  int64  
 12  totalTestResults          570400 non-null  int64  
 13  totalTestResultsIncrease  570400 non-null  i

In [8]:
# Create our features
#X = pd.get_dummies(df_equities, columns=["death", "deathIncrease", "hospitalizedIncrease", "hospitalizedCurrently", "negative", "positive", "totalTestResults", "totalTestResultsIncrease"])
X = df_equities.drop('p_status',axis=1)
# Create our target
y = pd.DataFrame(df_equities["p_status"])
X.head()

,high,iexClose,low,volume,death,deathIncrease,hospitalizedIncrease,hospitalizedCurrently,negative,negativeIncrease,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease
0,27.84,27.710,27.76,3283,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115
1,4.26,4.240,4.23,31824,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115
2,59.67,58.885,58.60,23896,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115
3,13.46,13.460,12.89,2365,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115
5,8.77,8.605,8.44,2894906,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115


In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 570400 entries, 0 to 631359
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   high                      570400 non-null  float64
 1   iexClose                  570400 non-null  float64
 2   low                       570400 non-null  float64
 3   volume                    570400 non-null  int64  
 4   death                     570400 non-null  int64  
 5   deathIncrease             570400 non-null  int64  
 6   hospitalizedIncrease      570400 non-null  int64  
 7   hospitalizedCurrently     570400 non-null  int64  
 8   negative                  570400 non-null  int64  
 9   negativeIncrease          570400 non-null  int64  
 10  positive                  570400 non-null  int64  
 11  positiveIncrease          570400 non-null  int64  
 12  totalTestResults          570400 non-null  int64  
 13  totalTestResultsIncrease  570400 non-null  i

In [10]:
X.describe()

,high,iexClose,low,volume,death,deathIncrease,hospitalizedIncrease,hospitalizedCurrently,negative,negativeIncrease,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease
count,570400.000000,570400.000000,570400.000000,5.704000e+05,570400.000000,570400.000000,570400.000000,570400.000000,5.704000e+05,570400.000000,5.704000e+05,570400.000000,5.704000e+05,5.704000e+05
mean,42.658513,44.234890,41.554962,9.234301e+05,221402.550000,1275.587500,2559.187500,56305.500000,3.922507e+07,250182.412500,9.540573e+06,99462.837500,1.438905e+08,1.276617e+06
std,107.594631,108.536875,104.526355,6.664651e+06,38625.202612,801.742567,1341.234424,29470.053439,9.169443e+06,124213.055681,3.574027e+06,68021.723619,4.674591e+07,4.017369e+05
min,0.000000,0.022900,0.000000,0.000000e+00,165088.000000,246.000000,-752.000000,28849.000000,2.446267e+07,-658774.000000,5.485765e+06,22310.000000,7.635677e+07,5.468590e+05
25%,8.160000,9.640000,7.805000,1.353775e+04,192707.250000,851.500000,1518.250000,32712.000000,3.177611e+07,212750.750000,6.823470e+06,43793.500000,1.053408e+08,9.472560e+05
50%,22.340000,23.460000,21.750000,9.192850e+04,212496.500000,1049.000000,2136.000000,41083.000000,3.823387e+07,260824.500000,8.174878e+06,63143.000000,1.356483e+08,1.169490e+06
75%,43.980000,45.480000,42.865000,4.433702e+05,242672.500000,1371.250000,3486.250000,79775.750000,4.651287e+07,304880.250000,1.158534e+07,161495.250000,1.777278e+08,1.639021e+06
max,4530.000000,4474.000000,4382.550000,1.864680e+09,317492.000000,3453.000000,5411.000000,119463.000000,5.674314e+07,456078.000000,1.836721e+07,236933.000000,2.412239e+08,2.010951e+06


In [11]:
# Check the balance of our target values
y['p_status'].value_counts()

1    314960
0    255440
Name: p_status, dtype: int64

In [12]:
y.head()

,p_status
0,1
1,0
2,1
3,1
5,1


In [13]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [14]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=200, random_state=78) 

In [16]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [17]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [18]:
predictions

array(['0', '0', '0', ..., '1', '1', '0'], dtype=object)

In [19]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, predictions)

0.5282653844605326

In [20]:
# Display the confusion matrix
confusion_matrix(y_test, predictions)

array([[26796, 37280],
       [28399, 50125]], dtype=int64)

In [21]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.20639424, 0.23271716, 0.20541893, 0.24388101, 0.0102327 ,
       0.0121559 , 0.01184118, 0.0108975 , 0.01026902, 0.01233703,
       0.0102261 , 0.01150768, 0.01021961, 0.01190195])

In [22]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.24388101486880567, 'volume'),
 (0.2327171584265753, 'iexClose'),
 (0.20639423604364684, 'high'),
 (0.20541893297913502, 'low'),
 (0.012337027120412757, 'negativeIncrease'),
 (0.012155899753590615, 'deathIncrease'),
 (0.01190194562891143, 'totalTestResultsIncrease'),
 (0.011841178116528276, 'hospitalizedIncrease'),
 (0.011507683736513693, 'positiveIncrease'),
 (0.0108974983861549, 'hospitalizedCurrently'),
 (0.010269019847264914, 'negative'),
 (0.010232699357882371, 'death'),
 (0.010226099766817447, 'positive'),
 (0.010219605967760679, 'totalTestResults')]

In [24]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.49      0.42      0.64      0.45      0.52      0.26     64076
          1       0.57      0.64      0.42      0.60      0.52      0.27     78524

avg / total       0.53      0.54      0.52      0.53      0.52      0.27    142600

